# Trying out sympy module

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import numpy as np
from Modules.sympy.classes import *
from Modules.sympy.utils import *
from sympy.physics.quantum import Dagger

In [3]:
Spin = RDBasis("\\sigma", 'spin', dim=2)
sx = Spin.project(sp.Matrix([[0, 1], [1, 0]]))
sz = Spin.project(sp.Matrix([[1, 0], [0, -1]]))

a = RDBoson("a", subspace ="b")
ad = RDBoson("{a^\\dagger}", subspace ="b", is_annihilation=False)

hbar = RDsymbol("hbar", order = 0)
omega = RDsymbol("omega", order = 0)
Omega_z = RDsymbol("\\Omega_{z}", order = 0)
g = RDsymbol("g", order = 1)

H = hbar * omega * ad * a + hbar * Omega_z * sp.Rational(1,2) * sz + hbar * g * (a + ad) * sx
H

\Omega_{z}*hbar*\sigma_{3}/2 + g*hbar*(a + {a^\dagger})*\sigma_{1} + hbar*omega*{a^\dagger}*a

In [4]:
group_by_diagonal(1 / (Omega_z**2 - omega**2))

{True: 1/(\Omega_{z}**2 - omega**2)}

In [5]:
Charge = RDBasis("Q", "charge", dim=2)
tx = Charge.project(sp.Matrix([[0, 1], [1, 0]]))

In [6]:
s1_symbols = [RDsymbol("S^1_{}".format(i), order=1) for i in range(4)]
s2_symbols = [RDsymbol("S^2_{}".format(i), order=1) for i in range(4)]
S1 = np.array(s1_symbols, dtype=object).dot(Spin._basis)
S2 = np.array(s2_symbols, dtype=object).dot(Spin._basis)
S = S1*ad + S2*a
S

(S^1_0*\sigma_{0} + S^1_1*\sigma_{1} + S^1_2*\sigma_{2} + S^1_3*\sigma_{3})*{a^\dagger} + (S^2_0*\sigma_{0} + S^2_1*\sigma_{1} + S^2_2*\sigma_{2} + S^2_3*\sigma_{3})*a

In [7]:
H0 = group_by_order(H)[0]
V = group_by_order(group_by_diagonal(H)[False])[1]
V

g*hbar*a*\sigma_{1} + g*hbar*{a^\dagger}*\sigma_{1}

In [8]:
sols = {}
for key, val in group_by_infinite_operators(Spin.project(expand_commutator(Commutator(H0, S) + V).doit())).items():
    if val == 0:
        continue
    display(key)
    display(val)
    sols.update(sp.solve(get_matrix(val), s1_symbols + s2_symbols))

{a^\dagger}

S^1_0*hbar*omega*\sigma_{0} - I*S^1_1*\Omega_{z}*hbar*\sigma_{2} + S^1_1*hbar*omega*\sigma_{1} + I*S^1_2*\Omega_{z}*hbar*\sigma_{1} + S^1_2*hbar*omega*\sigma_{2} + S^1_3*hbar*omega*\sigma_{3} + g*hbar*\sigma_{1}

a

-S^2_0*hbar*omega*\sigma_{0} - I*S^2_1*\Omega_{z}*hbar*\sigma_{2} - S^2_1*hbar*omega*\sigma_{1} + I*S^2_2*\Omega_{z}*hbar*\sigma_{1} - S^2_2*hbar*omega*\sigma_{2} - S^2_3*hbar*omega*\sigma_{3} + g*hbar*\sigma_{1}

In [9]:
group_by_infinite_operators(S.subs(sols))

{1: 0,
 a: I*\Omega_{z}*g*\sigma_{2}/(\Omega_{z}**2 - omega**2) - g*omega*\sigma_{1}/(\Omega_{z}**2 - omega**2),
 {a^\dagger}: I*\Omega_{z}*g*\sigma_{2}/(\Omega_{z}**2 - omega**2) + g*omega*\sigma_{1}/(\Omega_{z}**2 - omega**2)}

In [10]:
for key, val in group_by_infinite_operators(S.subs(sols)).items():
    if val == 0:
        continue
    display(val)
    display(get_matrix(val).simplify())

I*\Omega_{z}*g*\sigma_{2}/(\Omega_{z}**2 - omega**2) - g*omega*\sigma_{1}/(\Omega_{z}**2 - omega**2)

Matrix([
[                      0, g/(\Omega_{z} + omega)],
[-g/(\Omega_{z} - omega),                      0]])

I*\Omega_{z}*g*\sigma_{2}/(\Omega_{z}**2 - omega**2) + g*omega*\sigma_{1}/(\Omega_{z}**2 - omega**2)

Matrix([
[                      0, g/(\Omega_{z} - omega)],
[-g/(\Omega_{z} + omega),                      0]])

In [11]:
get_terms_and_factors(g*Omega_z/(omega**2 - Omega_z**2)*ad)

([\Omega_{z}*g*{a^\dagger}/(-\Omega_{z}**2 + omega**2)],
 [[\Omega_{z}, g, 1/(-\Omega_{z}**2 + omega**2), {a^\dagger}]])

In [12]:
group_by_diagonal(Spin.project(expand_commutator(H + Commutator(H, S) + sp.Rational(1,2) * Commutator(V, S)).doit().subs(sols).simplify().expand().nsimplify()))[False]

6*\Omega_{z}*g**2*hbar*a**2*\sigma_{3}/(2*\Omega_{z}**2 - 2*omega**2) + 6*\Omega_{z}*g**2*hbar*{a^\dagger}**2*\sigma_{3}/(2*\Omega_{z}**2 - 2*omega**2)

In [20]:
def get_ansatz(Vk):
    to_separate = list(group_by_infinite_operators(Vk ).keys())
    order = min(list(group_by_order(Vk).keys()))
    ansatz = 0
    symbol_s = np.array([RDsymbol("t")])
    for idx, term in enumerate(to_separate):
        tmp_s = np.array([RDsymbol(f"S^{idx}_{i}", order=order) for i in range(Spin._basis[0].dim**2)])
        symbol_s = np.concatenate((symbol_s, tmp_s))
        ansatz += term * (tmp_s * Spin._basis).sum()
    return ansatz, symbol_s[1:]


H_ordered = group_by_order(H)
H0 = H_ordered.get(0, 0)
Bk = 0

S = 0
H_final = H0
order = 4

for k in range(1, order+1):
    H_below_k = sum(H_ordered.get(j, 0) for j in range(k+1))
    Vk = group_by_diagonal(H_below_k)[False]
    S_k, symbols_sk = get_ansatz(Vk + Bk)

    S_k_grouped = group_by_infinite_operators(S_k)

    S_k_solved = 0

    expression_to_solve = Spin.project(expand_commutator(Commutator(H0, S_k) + Vk + Bk).doit()).simplify()
    sols = {}
    for key, val in group_by_infinite_operators(expression_to_solve).items():
        if val == 0:
            continue
        sols.update(sp.solve(get_matrix(val), list(symbols_sk))) # make get_ansatz return a list!!!
        
        for solution in symbols_sk: #big patch in case no solutions were found
            if solution not in sols:
                sols.update({solution: 0})
                
        S_k_solved += key * S_k_grouped[key].subs(sols) # RDOperator(S_k_grouped[key].name, "S", S_k_grouped[key].dim, S_k_grouped[key].matrix.subs(solution_sk)) 
    
        
    
    S += S_k_solved
    tmp_H = (1 / sp.factorial(k) * Spin.project(expand_commutator(nested_commutator(H_below_k, S, k)).doit())).simplify()

    temp_H_grouped = group_by_diagonal(tmp_H)
    H_final += group_by_order(temp_H_grouped[True]).get(k, 0)
    Bk = group_by_order(temp_H_grouped[False]).get(k+1, 0)

H_final

KeyError: a*{a^\dagger}*a

In [24]:
Vk + Bk

2*I*\Omega_{z}**3*g**3*hbar*omega*a*\sigma_{2}/(\Omega_{z}**4*omega**2 - 2*\Omega_{z}**2*omega**4 + omega**6) - 2*I*\Omega_{z}**3*g**3*hbar*omega*a**2*{a^\dagger}*\sigma_{2}/(\Omega_{z}**4*omega**2 - 2*\Omega_{z}**2*omega**4 + omega**6) - 2*I*\Omega_{z}**3*g**3*hbar*omega*a**3*\sigma_{2}/(\Omega_{z}**4*omega**2 - 2*\Omega_{z}**2*omega**4 + omega**6) + 2*I*\Omega_{z}**3*g**3*hbar*omega*{a^\dagger}*\sigma_{2}/(\Omega_{z}**4*omega**2 - 2*\Omega_{z}**2*omega**4 + omega**6) + 2*I*\Omega_{z}**3*g**3*hbar*omega*{a^\dagger}**2*a*\sigma_{2}/(\Omega_{z}**4*omega**2 - 2*\Omega_{z}**2*omega**4 + omega**6) + 2*I*\Omega_{z}**3*g**3*hbar*omega*{a^\dagger}**3*\sigma_{2}/(\Omega_{z}**4*omega**2 - 2*\Omega_{z}**2*omega**4 + omega**6) + 4*\Omega_{z}**2*g**3*hbar*omega**2*a*{a^\dagger}**2*\sigma_{1}/(\Omega_{z}**4*omega**2 - 2*\Omega_{z}**2*omega**4 + omega**6) + 28*\Omega_{z}**2*g**3*hbar*omega**2*a*\sigma_{1}/(\Omega_{z}**4*omega**2 - 2*\Omega_{z}**2*omega**4 + omega**6) - 24*\Omega_{z}**2*g**3*hbar*ome

In [29]:
group_by_infinite_operators(expression_to_solve)[a*ad*a].simplify()

2*hbar*omega*(-S^2_0*\Omega_{z}**4*\sigma_{0} + 2*S^2_0*\Omega_{z}**2*omega**2*\sigma_{0} - S^2_0*omega**4*\sigma_{0} - S^2_1*\Omega_{z}**4*\sigma_{1} + 2*S^2_1*\Omega_{z}**2*omega**2*\sigma_{1} - S^2_1*omega**4*\sigma_{1} - S^2_2*\Omega_{z}**4*\sigma_{2} + 2*S^2_2*\Omega_{z}**2*omega**2*\sigma_{2} - S^2_2*omega**4*\sigma_{2} - S^2_3*\Omega_{z}**4*\sigma_{3} + 2*S^2_3*\Omega_{z}**2*omega**2*\sigma_{3} - S^2_3*omega**4*\sigma_{3})/(\Omega_{z}**4 - 2*\Omega_{z}**2*omega**2 + omega**4)

In [31]:
sols

{S^0_1: 0,
 S^0_2: 0,
 S^0_0: 0,
 S^0_3: 0,
 S^1_0: 0,
 S^1_1: (2*\Omega_{z}**4*g**3 - \Omega_{z}**4*g*omega**2 - 34*\Omega_{z}**2*g**3*omega**2 + 2*\Omega_{z}**2*g*omega**4 - g*omega**6)/(\Omega_{z}**6*omega - 3*\Omega_{z}**4*omega**3 + 3*\Omega_{z}**2*omega**5 - omega**7),
 S^1_2: (I*\Omega_{z}**5*g + 26*I*\Omega_{z}**3*g**3 - 2*I*\Omega_{z}**3*g*omega**2 + 6*I*\Omega_{z}*g**3*omega**2 + I*\Omega_{z}*g*omega**4)/(\Omega_{z}**6 - 3*\Omega_{z}**4*omega**2 + 3*\Omega_{z}**2*omega**4 - omega**6),
 S^1_3: 0,
 S^2_0: 0,
 S^2_1: 0,
 S^2_2: 0,
 S^2_3: 0,
 S^3_0: 0,
 S^3_1: 0,
 S^3_2: 0,
 S^3_3: 0,
 S^4_0: 0,
 S^4_1: 0,
 S^4_2: 0,
 S^4_3: 0,
 S^5_0: 0,
 S^5_1: 0,
 S^5_2: 0,
 S^5_3: 0,
 S^6_0: 0,
 S^6_1: 0,
 S^6_2: 0,
 S^6_3: 0,
 S^7_0: 0,
 S^7_1: 0,
 S^7_2: 0,
 S^7_3: 0,
 S^8_0: 0,
 S^8_1: 0,
 S^8_2: 0,
 S^8_3: 0,
 S^9_0: 0,
 S^9_1: 0,
 S^9_2: 0,
 S^9_3: 0}